<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/LSTM_pilkottu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 0. Esitiedot ja Tavoite (LSTM v1 - Uusi Data)
"""
Otsoniennuste Helsinki - LSTM-malli v1 (Uudella Datalla)

Tavoite:
1. Ladata uusi esikäsitelty data Parquet-tiedostosta (sis. pilvisyys).
2. Esikäsitellä data neuroverkkomallille sopivaksi (kuten aiemmassa GRU v3:ssa).
3. Määritellä ja kouluttaa LSTM (Long Short-Term Memory) -malli PyTorchilla.
4. Tehdä ennusteita testijaksolle.
5. Arvioida mallin suorituskykyä (RMSE, MAE, baseline-vertailu, 8h raja)
   ja verrata sitä aiempiin GRU-, ARIMA- ja SARIMAX-tuloksiin.
6. Visualisoida ennusteita.

Käyttää refaktoroitua asetustiedostoa (Osa 1).
"""
print("Osa 0: Esitiedot (LSTM v1) - OK")

In [ ]:
# @title 1. Tuonnit ja Asetukset (Korjattu & Selkeytetty RNN/LSTM varten)

# Peruskirjastot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import os
import math
import copy
import traceback
import warnings

# Sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report

# Statsmodels (Vain ADF-testi ja kuvaajat tarvitaan nyt)
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# SARIMAXia ei tuoda tässä skriptissä

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Muut
from tqdm.notebook import tqdm

# --- Yleisasetukset ---
warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
print(f"Käytettävä laite: {device}")

# --- Data-asetukset ---
BASE_GITHUB_URL = 'https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/'
PARQUET_PATH = 'data/processed/processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet'
DATA_URL = BASE_GITHUB_URL + PARQUET_PATH
LOCAL_DATA_PATH = 'processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet'
TARGET_COLUMN = 'Otsoni [µg/m³]'
ALL_COLUMNS_IN_PARQUET = [
    'Otsoni [µg/m³]',
    'Lämpötilan keskiarvo [°C]',
    'Keskituulen nopeus [m/s]',
    'Ilmanpaineen keskiarvo [hPa]',
    'Tuulen suunnan keskiarvo [°]',
    'Pilvisyys [okta]'
]
# EXOG_COLUMNS_BASE ei tarvita suoraan RNN-malleille tällä tavalla,
# koska kaikki ominaisuudet menevät INPUT_SIZEen.

# --- Ennustus- ja Jakoasetukset ---
FORECAST_HORIZON = 24 # Tuntia
TEST_SPLIT_RATIO = 0.15
VALID_SPLIT_RATIO = 0.15

# --- RNN/LSTM/GRU Mallin Hyperparametrit ---
RNN_HYPERPARAMS = {
    'model_type': 'LSTM',  # Vaihda 'GRU' tarvittaessa
    'input_size': None,    # Lasketaan Osa 3:ssa datan käsittelyn jälkeen
    'hidden_size': 64,
    'num_layers': 2,
    'output_size': FORECAST_HORIZON, # = 24
    'dropout_prob': 0.2
}

# --- Neuroverkkojen Koulutusparametrit ---
TRAIN_HYPERPARAMS = {
    'batch_size': 64,
    'learning_rate': 0.001,
    'epochs': 75,
    'patience': 10 # Early stopping
}

# --- Arviointiasetukset ---
O3_THRESHOLD_8H_AVG = 85 # µg/m³ (Käytetään testattua alempaa rajaa)

print("\nOsa 1: Tuonnit ja Asetukset (Korjattu RNN/LSTM) - OK")
print(f"Kohdemuuttuja: {TARGET_COLUMN}")
print(f"Parquet-polku: {PARQUET_PATH}")
print(f"Käytettävä RNN-tyyppi: {RNN_HYPERPARAMS['model_type']}")
print(f"8h Keskiarvon kynnysarvo: {O3_THRESHOLD_8H_AVG} µg/m³")

In [ ]:
# @title 2. Funktiot Datan Lataukseen ja Käsittelyyn (GRU v3 / LSTM v1) - KORJATTU NaT KÄSITTELY

import pandas as pd
import numpy as np
import requests
import io
import os
import re
import traceback
# Varmistetaan StandardScaler tuonti
try:
    from sklearn.preprocessing import StandardScaler
except ImportError:
     print("VIRHE: scikit-learn ei ole asennettu tai sitä ei voitu tuoda.")
     StandardScaler = None


# Funktio datan lataamiseen URL:sta (jos paikallista ei löydy)
def download_data(url, local_path):
    """Lataa tiedoston URL:sta paikalliseen polkuun."""
    try:
        print(f"Yritetään ladata dataa osoitteesta {url}...")
        response = requests.get(url)
        response.raise_for_status()
        target_dir = os.path.dirname(local_path)
        if target_dir and not os.path.exists(target_dir):
             os.makedirs(target_dir, exist_ok=True)
             print(f"Luotiin hakemisto: {target_dir}")
        with open(local_path, 'wb') as f:
            f.write(response.content)
        print(f"Data ladattu onnistuneesti: {local_path}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"Virhe datan latauksessa: {e}")
        return False
    except Exception as e:
         print(f"Odottamaton virhe tiedoston tallennuksessa polkuun {local_path}: {e}")
         traceback.print_exc()
         return False

# Funktio Parquet-datan lataamiseen
def load_parquet_data(filepath_or_url, local_cache_path=LOCAL_DATA_PATH):
    """Lataa datan Parquet-tiedostosta (paikallisesti tai URL:sta, käyttää välimuistia). KORJATTU NaT-käsittely."""
    filepath_to_read = None
    if filepath_or_url.startswith('http'):
         if os.path.exists(local_cache_path):
              print(f"Käytetään paikallista välimuistitiedostoa: {local_cache_path}")
              filepath_to_read = local_cache_path
         else:
              print(f"Paikallista tiedostoa {local_cache_path} ei löytynyt.")
              if download_data(filepath_or_url, local_cache_path):
                   filepath_to_read = local_cache_path
              else: return None
    else:
         filepath_to_read = filepath_or_url
         if not os.path.exists(filepath_to_read):
              print(f"VIRHE: Annettua paikallista tiedostoa {filepath_to_read} ei löytynyt.")
              return None

    if filepath_to_read:
        try:
            print(f"Ladataan dataa tiedostosta: {filepath_to_read}")
            df = pd.read_parquet(filepath_to_read)
            print(f"Parquet-tiedosto ladattu, muoto: {df.shape}")

            # --- Aikaindeksin varmistus ja käsittely ---
            if 'Aikaleima' not in df.columns and not isinstance(df.index, pd.DatetimeIndex):
                 print("Indeksi ei ole DatetimeIndex, eikä 'Aikaleima'-sarake löydy. Yritetään muuntaa indeksi...")
                 try: df.index = pd.to_datetime(df.index); print("Indeksi muunnettu datetime-indeksiksi.")
                 except Exception as e_idx: print(f"VIRHE: Indeksin muunto datetimeksi epäonnistui: {e_idx}"); return None
            elif 'Aikaleima' in df.columns:
                 print("Käytetään 'Aikaleima'-saraketta indeksinä..."); df['Aikaleima'] = pd.to_datetime(df['Aikaleima']); df.set_index('Aikaleima', inplace=True); print("'Aikaleima' asetettu indeksiksi.")

            if not isinstance(df.index, pd.DatetimeIndex): print("VIRHE: Indeksi ei ole DatetimeIndex käsittelyn jälkeen."); return None

            # Varmista aikavyöhyke
            if df.index.tz is None:
                 print("Asetetaan aikavyöhykkeeksi Europe/Helsinki...")
                 try:
                     df = df.tz_localize('Europe/Helsinki', ambiguous='NaT', nonexistent='NaT')
                     print("Aikavyöhyke asetettu (ambiguous/nonexistent='NaT').")
                 except Exception as e_tz: print(f"VAROITUS: Aikavyöhykkeen asetus epäonnistui: {e_tz}. Jatketaan naiivilla indeksillä.")
            else: print(f"Aikavyöhyke on jo asetettu: {df.index.tz}")

            # *** KORJATTU NaT-käsittely ***
            nat_index_rows = df.index.isnull()
            if nat_index_rows.any():
                rows_before_nat = len(df)
                print(f"VAROITUS: Löydettiin {nat_index_rows.sum()} virheellistä aikaleimaa (NaT) indeksistä. Poistetaan rivit...")
                df = df[~nat_index_rows] # Valitse rivit, joiden indeksi EI ole NaT
                print(f"Poistettu {rows_before_nat - len(df)} NaT-indeksiriviä.")

            df.sort_index(inplace=True)
            print(f"Datan perustiedot: Aikaväli=[{df.index.min()} - {df.index.max()}]")
            print(f"Sarakkeet: {df.columns.tolist()}")

            # Tarkista NaN-arvot sisällössä
            if df.isnull().any().any():
                print("VAROITUS: Datassa NaN-arvoja latauksen/aikakäsittelyn jälkeen. Yritetään täyttää ffill/bfill...")
                df.ffill(inplace=True); df.bfill(inplace=True)
                if df.isnull().any().any(): print("VIRHE: Ei voitu täyttää kaikkia NaN-arvoja soluista."); df.dropna(inplace=True)
                print(f"Datan muoto NaN-käsittelyn jälkeen: {df.shape}")

            return df

        except Exception as e:
            print(f"VIRHE Parquet-tiedoston lukemisessa tai aikakäsittelyssä ('{filepath_to_read}'): {e}")
            traceback.print_exc(); return None
    else: print("VIRHE: Tiedostopolkua lukemiseen ei saatu määritettyä."); return None


# Funktio ominaisuuksien muokkaukseen (feature engineering) neuroverkolle
def feature_engineer_gru(df):
    """Lisää aika- ja syklisiä ominaisuuksia neuroverkkokäyttöön."""
    print("\nSuoritetaan ominaisuuksien muokkaus (Feature Engineering)...")
    if not isinstance(df, pd.DataFrame): print("VIRHE: feature_engineer_gru syöte ei DataFrame."); return None
    df_eng = df.copy()
    if not isinstance(df_eng.index, pd.DatetimeIndex): print("VIRHE: Indeksi ei ole DatetimeIndex FE:ssä."); return df_eng

    try: # Kellonaika
        df_eng['hour_sin'] = np.sin(2 * np.pi * df_eng.index.hour / 24.0)
        df_eng['hour_cos'] = np.cos(2 * np.pi * df_eng.index.hour / 24.0)
        print("Lisätty syklinen kellonaika (hour_sin, hour_cos).")
    except Exception as e: print(f"VIRHE kellonaikaominaisuuksien luonnissa: {e}")

    wind_dir_col_orig = 'Tuulen suunnan keskiarvo [°]'
    if wind_dir_col_orig in df_eng.columns: # Tuulen suunta
        try:
            df_eng[wind_dir_col_orig] = pd.to_numeric(df_eng[wind_dir_col_orig], errors='coerce')
            if df_eng[wind_dir_col_orig].isnull().any(): print(f"VAROITUS: NaN '{wind_dir_col_orig}':ssa. Täytetään."); df_eng[wind_dir_col_orig]=df_eng[wind_dir_col_orig].ffill().bfill()
            df_eng.dropna(subset=[wind_dir_col_orig], inplace=True)
            if not df_eng.empty: # Varmista ettei kaikki mennyt NaNnien poistossa
                 df_eng['wind_dir_rad'] = np.deg2rad(df_eng[wind_dir_col_orig])
                 df_eng['wind_dir_sin'] = np.sin(df_eng['wind_dir_rad'])
                 df_eng['wind_dir_cos'] = np.cos(df_eng['wind_dir_rad'])
                 df_eng.drop(columns=[wind_dir_col_orig, 'wind_dir_rad'], inplace=True, errors='ignore')
                 print("Muunnettu tuulen suunta sykliseksi (wind_dir_sin, wind_dir_cos).")
            else: print(f"Data tyhjeni NaN-poiston jälkeen sarakkeelle '{wind_dir_col_orig}'.")
        except Exception as e: print(f"VIRHE tuulensuuntaominaisuuksien luonnissa: {e}"); df_eng.drop(columns=['wind_dir_rad','wind_dir_sin','wind_dir_cos'],inplace=True,errors='ignore')
    else: print(f"Saraketta '{wind_dir_col_orig}' ei löytynyt.")

    print("Ominaisuuksien muokkaus valmis.")
    return df_eng


# Funktio sekvenssien luontiin PyTorchille
def create_sequences_pytorch(features_scaled, targets_original, sequence_length, prediction_horizon):
    """Luo syötesekvenssejä (X, skaalattu) ja alkuperäisiä kohde-ennusteita (y_orig)."""
    X, y_orig = [], []
    print(f"\nLuodaan PyTorch-sekvenssejä: sequence_length={sequence_length}, prediction_horizon={prediction_horizon}")
    print(f"features_scaled shape: {features_scaled.shape}, targets_original shape: {targets_original.shape}")
    required_len = sequence_length + prediction_horizon
    if len(features_scaled) < required_len: print(f"VAROITUS: Ei tarpeeksi dataa ({len(features_scaled)}). Tarvitaan {required_len}."); return np.array(X), np.array(y_orig)
    if targets_original.ndim == 1: targets_original = targets_original.reshape(-1, 1); print(f"Muutettu targets_original muotoon: {targets_original.shape}")
    for i in range(len(features_scaled) - required_len + 1):
        X.append(features_scaled[i:(i + sequence_length)])
        y_orig.append(targets_original[i + sequence_length : i + sequence_length + prediction_horizon, 0])
    print(f"Luotu {len(X)} sekvenssiä.")
    X = np.array(X); y_orig = np.array(y_orig)
    if y_orig.ndim == 2: y_orig = y_orig[..., np.newaxis]; print(f"Muutettu y_orig muotoon: {y_orig.shape}")
    elif y_orig.size > 0: print(f"VAROITUS: y_orig muoto ({y_orig.shape}) ei ole odotettu (samples, horizon, 1).")
    return X, y_orig

print("\nOsa 2: Funktiot datan käsittelyyn (GRU v3 / LSTM v1) - KORJATTU - OK")

In [ ]:
# @title 3. Pääskriptin Suoritus: Datan Käsittely (GRU v3 / LSTM v1) - LISÄTTY PARAMETRIVARMISTUS

# Tässä solussa kutsutaan Osassa 2 määriteltyjä funktioita
# ja valmistellaan data DataLoadereihin asti.

import pandas as pd
import numpy as np
import traceback
# Varmistetaan tuonnit uudelleen solun ajon varalta
try:
    import torch
    from torch.utils.data import TensorDataset, DataLoader
except ImportError: raise ImportError("PyTorch ei ladattu.")
try:
    from sklearn.preprocessing import StandardScaler
except ImportError: raise ImportError("scikit-learn ei ladattu.")


print("--- Aloitetaan Datan Käsittely Neuroverkkoa varten ---")

# Alustetaan muuttujat
df_raw_full = None; df_raw = None; df_engineered = None
train_loader = None; valid_loader = None; test_loader = None
feature_scaler = None; o3_scaler = None
INPUT_SIZE = None
test_timestamps = None
X_train, y_train_original, X_valid, y_valid_original, X_test, y_test_original = [None]*6
y_train_scaled, y_valid_scaled = None, None


try:
    # *** LISÄTTY VARMISTUS PARAMETREILLE TÄSSÄ SOLUSSA ***
    # Varmistetaan, että nämä ovat määriteltyjä ennen käyttöä
    if 'SEQUENCE_LENGTH' not in locals() or SEQUENCE_LENGTH is None:
        print("VAROITUS: SEQUENCE_LENGTH ei löytynyt/ollut määritelty. Asetetaan oletusarvo 72.")
        SEQUENCE_LENGTH = 72
    if 'PREDICTION_HORIZON' not in locals() or PREDICTION_HORIZON is None:
        print("VAROITUS: PREDICTION_HORIZON ei löytynyt/ollut määritelty. Asetetaan oletusarvo 24.")
        PREDICTION_HORIZON = 24
    if 'TARGET_COLUMN' not in locals() or TARGET_COLUMN is None:
         # Haetaan Osasta 1, jos mahdollista, muuten asetetaan oletus
         try: TARGET_COLUMN = TARGET_COLUMN # Yritä käyttää globaalia
         except NameError: TARGET_COLUMN = 'Otsoni [µg/m³]'; print("Asetettiin TARGET_COLUMN oletusarvoon.")
    if 'ALL_COLUMNS_IN_PARQUET' not in locals() or ALL_COLUMNS_IN_PARQUET is None:
         # Asetetaan oletus Osan 1 perusteella
         ALL_COLUMNS_IN_PARQUET = ['Otsoni [µg/m³]', 'Lämpötilan keskiarvo [°C]', 'Keskituulen nopeus [m/s]', 'Ilmanpaineen keskiarvo [hPa]', 'Tuulen suunnan keskiarvo [°]', 'Pilvisyys [okta]']
         print("Asetettiin ALL_COLUMNS_IN_PARQUET oletusarvoon.")
    if 'TRAIN_HYPERPARAMS' not in locals() or TRAIN_HYPERPARAMS is None:
         TRAIN_HYPERPARAMS = {'batch_size': 64} # Tarvitaan vain batch_size DataLoadereihin tässä solussa
         print("Asetettiin TRAIN_HYPERPARAMS['batch_size'] oletusarvoon.")
    elif 'batch_size' not in TRAIN_HYPERPARAMS:
          TRAIN_HYPERPARAMS['batch_size'] = 64
          print("Asetettiin TRAIN_HYPERPARAMS['batch_size'] oletusarvoon.")


    print(f"\nKäytetään Osassa 3: SEQUENCE_LENGTH={SEQUENCE_LENGTH}, PREDICTION_HORIZON={PREDICTION_HORIZON}, BATCH_SIZE={TRAIN_HYPERPARAMS['batch_size']}")
    # ******************************************************

    # 1. Lataa data käyttäen Osa 2:n funktiota
    df_raw_full = load_parquet_data(DATA_URL) # Käyttää DATA_URL ja LOCAL_DATA_PATH Osa 1:stä
    if df_raw_full is None: raise ValueError("Datan lataus epäonnistui (load_parquet_data palautti None).")
    if df_raw_full.empty: raise ValueError("Ladattu DataFrame on tyhjä.")

    # 2. Tarkista ja valitse alkuperäiset sarakkeet
    print("\nTarkistetaan ja valitaan alkuperäiset sarakkeet...")
    missing_cols_raw = [col for col in ALL_COLUMNS_IN_PARQUET if col not in df_raw_full.columns]
    if missing_cols_raw:
        raise ValueError(f"Seuraavat määritellyt sarakkeet puuttuvat Parquet-tiedostosta: {missing_cols_raw}")
    df_raw = df_raw_full[ALL_COLUMNS_IN_PARQUET].copy()
    print("Alkuperäiset sarakkeet valittu.")

    # 3. Suorita Feature Engineering
    df_engineered = feature_engineer_gru(df_raw)
    if df_engineered is None: raise ValueError("Ominaisuuksien muokkaus (feature_engineer_gru) epäonnistui.")
    if df_engineered.empty: raise ValueError("DataFrame tyhjä feature engineeringin jälkeen.")

    # 4. Määritä lopulliset ominaisuudet ja INPUT_SIZE
    FINAL_FEATURE_COLUMNS = df_engineered.columns.tolist()
    if TARGET_COLUMN not in FINAL_FEATURE_COLUMNS: print(f"VAROITUS: Kohde '{TARGET_COLUMN}' ei lopullisissa ominaisuuksissa!")
    INPUT_SIZE = len(FINAL_FEATURE_COLUMNS)
    if INPUT_SIZE == 0: raise ValueError("Lopullisia ominaisuuksia ei löytynyt feature engineeringin jälkeen.")
    print(f"\nLopullinen ominaisuuksien määrä (INPUT_SIZE): {INPUT_SIZE}")
    print(f"Lopulliset ominaisuudet mallille: {FINAL_FEATURE_COLUMNS}")

    # 5. Jaa data harjoitus-, validointi- ja testijoukkoihin
    n = len(df_engineered)
    # Käytetään nyt varmistettuja arvoja
    min_data_len_needed = (SEQUENCE_LENGTH + PREDICTION_HORIZON) * 3
    if n < min_data_len_needed: print(f"VAROITUS: Datan pituus ({n}) voi olla liian lyhyt jakoon ja sekvenssien luontiin (tarvitaan > {min_data_len_needed}).")
    test_split_idx = int(n * (1 - TEST_SPLIT_RATIO))
    valid_split_idx = int(test_split_idx * (1 - VALID_SPLIT_RATIO / (1 - TEST_SPLIT_RATIO)))
    df_train = df_engineered[:valid_split_idx]
    df_valid = df_engineered[valid_split_idx:test_split_idx]
    df_test = df_engineered[test_split_idx:]
    min_len_for_seq = SEQUENCE_LENGTH + PREDICTION_HORIZON
    if len(df_train) < min_len_for_seq or len(df_valid) < min_len_for_seq or len(df_test) < min_len_for_seq:
        print(f"Train={len(df_train)}, Valid={len(df_valid)}, Test={len(df_test)}, Min={min_len_for_seq}")
        raise ValueError("Liian vähän dataa yhdessä tai useammassa jaossa sekvenssien luontia varten.")
    print(f"\nDatan jako: Train={len(df_train)}, Valid={len(df_valid)}, Test={len(df_test)}")

    # 6. Skaalaa data
    print("\nSkaalataan dataa...")
    feature_scaler = StandardScaler()
    scaled_train_features = feature_scaler.fit_transform(df_train)
    scaled_valid_features = feature_scaler.transform(df_valid)
    scaled_test_features = feature_scaler.transform(df_test)
    print("Ominaisuudet skaalattu.")
    o3_scaler = StandardScaler()
    o3_scaler.fit(df_raw.loc[df_train.index, [TARGET_COLUMN]])
    print("Kohdemuuttuja (O3) skaalain sovitettu.")

    # 7. Hae alkuperäiset O3-kohdearvot
    o3_train_targets_original = df_raw.loc[df_train.index, [TARGET_COLUMN]].values
    o3_valid_targets_original = df_raw.loc[df_valid.index, [TARGET_COLUMN]].values
    o3_test_targets_original = df_raw.loc[df_test.index, [TARGET_COLUMN]].values

    # 8. Luo sekvenssit
    X_train, y_train_original = create_sequences_pytorch(scaled_train_features, o3_train_targets_original, SEQUENCE_LENGTH, PREDICTION_HORIZON)
    X_valid, y_valid_original = create_sequences_pytorch(scaled_valid_features, o3_valid_targets_original, SEQUENCE_LENGTH, PREDICTION_HORIZON)
    X_test, y_test_original = create_sequences_pytorch(scaled_test_features, o3_test_targets_original, SEQUENCE_LENGTH, PREDICTION_HORIZON)
    if X_train.size == 0 or X_valid.size == 0 or X_test.size == 0: raise ValueError("Sekvenssien luonti epäonnistui (yksi tai useampi X on tyhjä).")

    # 9. Skaalaa kohdesekvenssit
    y_train_scaled = o3_scaler.transform(y_train_original.reshape(-1, 1)).reshape(y_train_original.shape)
    y_valid_scaled = o3_scaler.transform(y_valid_original.reshape(-1, 1)).reshape(y_valid_original.shape)
    print("Kohdesekvenssit skaalattu.")

    # 10. Muunna Tensoreiksi
    print("\nMuunnetaan data PyTorch-tensoreiksi...")
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32); y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
    X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32); y_valid_tensor = torch.tensor(y_valid_scaled, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32); y_test_tensor_original = torch.tensor(y_test_original, dtype=torch.float32)
    print("Tensorit luotu.")

    # 11. Luo DataLoaderit
    # drop_last=True harjoitusdatalle voi auttaa jos viimeinen erä pieni
    batch_size = TRAIN_HYPERPARAMS.get('batch_size', 64) # Hae batch_size asetuksista
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor); train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor); valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor_original); test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    print("DataLoaderit luotu.")

    # 12. Tallenna testiaikaleimat
    print("\nTallennetaan testiaikaleimoja...")
    if 'df_test' in locals() and not df_test.empty and len(df_test) >= SEQUENCE_LENGTH and 'X_test' in locals() and len(X_test) > 0:
        try:
            test_start_index_loc = df_engineered.index.get_loc(df_test.index[0])
            test_start_indices = test_start_index_loc + SEQUENCE_LENGTH
            end_index = test_start_indices + len(X_test)
            if end_index <= len(df_engineered.index):
                test_timestamps = df_engineered.index[test_start_indices : end_index]
                print(f"Testiaikaleimat tallennettu ({len(test_timestamps)} kpl).")
            else: print("VAROITUS: Ei voitu määrittää kaikkia testiaikaleimoja."); test_timestamps = None
        except Exception as e_ts: print(f"VIRHE testiaikaleimojen tallennuksessa: {e_ts}"); test_timestamps = None
    else: print("VAROITUS: Testidata liian lyhyt/tyhjä aikaleimoille."); test_timestamps = None

    print(f"\nLopulliset muodot DataLoadereihin menevälle datalle:")
    print(f"X_train_tensor: {X_train_tensor.shape}, y_train_tensor: {y_train_tensor.shape}")
    print(f"X_valid_tensor: {X_valid_tensor.shape}, y_valid_tensor: {y_valid_tensor.shape}")
    print(f"X_test_tensor:  {X_test_tensor.shape}, y_test_tensor_original (tensor): {y_test_tensor_original.shape}")

    print("\nOsa 3: Datan käsittely - VALMIS")

# Käsittele mahdolliset virheet päälohkossa
except ValueError as ve:
     print(f"\n---> VIRHE DATAN KÄSITTELYSSÄ (ValueError): {ve} <---")
     traceback.print_exc()
     train_loader, valid_loader, test_loader, INPUT_SIZE = [None]*4 # Nollaa kriittiset muuttujat
except KeyError as ke:
     print(f"\n---> VIRHE DATAN KÄSITTELYSSÄ (KeyError): Saraketta {ke} ei löytynyt <---")
     traceback.print_exc()
     train_loader = None; valid_loader = None; test_loader = None; INPUT_SIZE=None;
except Exception as e:
     print(f"\n---> ODOTTAMATON VIRHE DATAN KÄSITTELYSSÄ (Osa 3) <---")
     traceback.print_exc()
     train_loader = None; valid_loader = None; test_loader = None; INPUT_SIZE=None;

In [ ]:
# @title 4. LSTM-Mallin Määrittely (LSTM v1)

import torch
import torch.nn as nn
import traceback # Virheiden jäljitykseen

print("--- Määritellään LSTM-malli ---")

# Varmistetaan tarvittavat parametrit globaalista scopesta (asetettu Osassa 1 & 3)
model_params_ok = True
# Käytetään try-exceptia, koska muuttujat voivat puuttua, jos soluja ajetaan epäjärjestyksessä
try:
    # Haetaan arvot aiemmin määritellyistä muuttujista/sanakirjoista
    input_size = INPUT_SIZE
    hidden_size = RNN_HYPERPARAMS['hidden_size']
    num_layers = RNN_HYPERPARAMS['num_layers']
    output_size = RNN_HYPERPARAMS['output_size']
    dropout_prob = RNN_HYPERPARAMS['dropout_prob']
    # Tulostetaan käytettävät arvot
    print(f"Käytetään LSTM-parametreja: Input={input_size}, Hidden={hidden_size}, Layers={num_layers}, Output={output_size}, Dropout={dropout_prob}")
    # Tarkistetaan arvojen järkevyys
    assert all(isinstance(v, int) and v > 0 for v in [input_size, hidden_size, num_layers, output_size]), "Kokoarvojen oltava positiivisia kokonaislukuja."
    assert isinstance(dropout_prob, float) and 0.0 <= dropout_prob < 1.0, "Dropout oltava [0, 1)."

except (NameError, KeyError, TypeError, AssertionError) as param_err:
     print(f"VIRHE hyperparametreissa: {param_err}. Varmista, että Osat 1 ja 3 on ajettu onnistuneesti.")
     model_params_ok = False # Estetään luokan määrittely


# Määritellään malliluokka vain jos parametrit ok
if model_params_ok:
    class LSTMModel(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
            super(LSTMModel, self).__init__()
            self.hidden_size = hidden_size
            self.num_layers = num_layers
            lstm_dropout = dropout_prob if num_layers > 1 else 0.0
            self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                                batch_first=True, dropout=lstm_dropout)
            self.fc = nn.Linear(hidden_size, output_size)

        def forward(self, x):
            # x: (batch, seq_len, input_size)
            # Alustetaan h0 ja c0 oikealle laitteelle
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
            # LSTM ulostulo
            out, _ = self.lstm(x, (h0, c0))
            # Otetaan viimeinen aika-askel
            out = out[:, -1, :] # (batch, hidden)
            # Lineaarinen kerros
            out = self.fc(out) # (batch, output_size)
            return out

    # Testataan luokan alustus nopeasti
    try:
         temp_model_test = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout_prob)
         print("\nLSTMModel-luokka määritelty ja alustus testattu onnistuneesti.")
         print("Mallin rakenne:")
         print(temp_model_test)
         del temp_model_test
    except Exception as e_model_def:
         print(f"VIRHE LSTMModel-luokan alustuksessa: {e_model_def}")
         traceback.print_exc()
         raise RuntimeError("Mallin määrittely epäonnistui.")

    print("\nOsa 4: LSTM-Mallin Määrittely (LSTM v1) - OK")

else:
     print("\nOsa 4: LSTM-Mallin Määrittely - EPÄONNISTUI puuttuvien/virheellisten parametrien vuoksi.")

In [ ]:
# @title 5. Koulutusfunktion Määrittely (LSTM v1)

# Tämä funktio on sama kuin aiemmassa GRU-skriptissä

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm # Edistymispalkki
import copy # Mallin tilan kopiointiin
import traceback

print("--- Määritellään train_model -funktio ---")

# Varmistetaan tarvittavien luokkien olemassaolo
if 'DataLoader' not in locals(): raise NameError("DataLoader ei ole määritelty.")
if 'nn' not in locals() or 'optim' not in locals(): raise NameError("torch.nn tai torch.optim ei ole tuotu.")

def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs, device, patience):
    """Kouluttaa mallin ja käyttää Early Stoppingia (sis. .squeeze(-1) korjauksen)."""

    # Alkutarkistukset
    if not isinstance(model, nn.Module): raise TypeError("Vaaditaan PyTorch-malli.")
    if not isinstance(train_loader, DataLoader): raise TypeError("Vaaditaan DataLoader train_loaderille.")
    if not isinstance(valid_loader, DataLoader): raise TypeError("Vaaditaan DataLoader valid_loaderille.")
    if not isinstance(criterion, nn.modules.loss._Loss): raise TypeError("Vaaditaan PyTorch häviöfunktio.")
    if not isinstance(optimizer, optim.Optimizer): raise TypeError("Vaaditaan PyTorch optimoija.")
    if not isinstance(epochs, int) or epochs <= 0: raise ValueError("Epochs oltava positiivinen kokonaisluku.")
    if not isinstance(patience, int) or patience <= 0: raise ValueError("Patience oltava positiivinen kokonaisluku.")

    train_losses = []
    valid_losses = []
    best_valid_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    print(f"\nAloitetaan koulutus {epochs} epochilla...")
    print(f"Early stopping -kärsivällisyys: {patience} epochia.")

    for epoch in tqdm(range(epochs), desc="Epochs"):
        model.train() # Koulutustila
        running_train_loss = 0.0
        batch_count = 0
        try: # Try-except epochin sisällä
            for inputs, targets_scaled in train_loader:
                batch_count += 1
                inputs, targets_scaled = inputs.to(device), targets_scaled.to(device)

                optimizer.zero_grad()
                outputs_scaled = model(inputs) # Ennusteet, muoto (batch, 24)

                # --- Käsittele kohdemuoto (squeeze) ---
                if targets_scaled.ndim == outputs_scaled.ndim + 1 and targets_scaled.shape[-1] == 1:
                    targets_squeezed = targets_scaled.squeeze(-1) # Muoto (batch, 24)
                elif targets_scaled.shape == outputs_scaled.shape:
                    targets_squeezed = targets_scaled # Muodot täsmäävät jo
                else:
                     print(f"\nVIRHE Epoch {epoch+1}, Batch {batch_count} (Train): Muodot eivät täsmää loss-laskentaa varten!")
                     print(f"Output shape: {outputs_scaled.shape}, Target shape: {targets_scaled.shape}")
                     raise RuntimeError("Muodot eivät täsmää loss-laskentaa varten")

                loss = criterion(outputs_scaled, targets_squeezed)
                loss.backward()
                optimizer.step()
                running_train_loss += loss.item() * inputs.size(0)

            epoch_train_loss = running_train_loss / len(train_loader.dataset) if len(train_loader.dataset) > 0 else 0
            train_losses.append(epoch_train_loss)

        except Exception as e_train_epoch:
             print(f"\nVIRHE koulutusloopissa (Epoch {epoch+1}, Batch {batch_count}): {e_train_epoch}")
             traceback.print_exc()
             print("Keskeytetään koulutus.")
             model = None; return model, train_losses, valid_losses

        # --- Validointivaihe ---
        model.eval()
        running_valid_loss = 0.0
        valid_batch_count = 0
        try: # Try-except validointiin
            with torch.no_grad():
                for inputs, targets_scaled in valid_loader:
                    valid_batch_count += 1
                    inputs, targets_scaled = inputs.to(device), targets_scaled.to(device)
                    outputs_scaled = model(inputs)

                    # --- Käsittele kohdemuoto (squeeze) ---
                    if targets_scaled.ndim == outputs_scaled.ndim + 1 and targets_scaled.shape[-1] == 1:
                        targets_squeezed = targets_scaled.squeeze(-1)
                    elif targets_scaled.shape == outputs_scaled.shape:
                        targets_squeezed = targets_scaled
                    else:
                         print(f"\nVIRHE Epoch {epoch+1} (Valid), Batch {valid_batch_count}: Muodot eivät täsmää loss-laskentaa varten!")
                         print(f"Output shape: {outputs_scaled.shape}, Target shape: {targets_scaled.shape}")
                         raise RuntimeError("Muodot eivät täsmää validointi loss-laskentaa varten")

                    loss = criterion(outputs_scaled, targets_squeezed)
                    running_valid_loss += loss.item() * inputs.size(0)

            epoch_valid_loss = running_valid_loss / len(valid_loader.dataset) if len(valid_loader.dataset) > 0 else 0
            valid_losses.append(epoch_valid_loss)

            print(f"Epoch {epoch+1:02d}/{epochs} - Train Loss: {epoch_train_loss:.6f} - Valid Loss: {epoch_valid_loss:.6f}", end="")

            # Early Stopping Check
            if epoch_valid_loss < best_valid_loss:
                best_valid_loss = epoch_valid_loss
                epochs_no_improve = 0
                try: best_model_state = copy.deepcopy(model.state_dict()); print(" (Uusi paras!)")
                except Exception as e_state: print(f" (VIRHE tilan tallennuksessa: {e_state})"); best_model_state = None
            else:
                epochs_no_improve += 1
                print(f" (Ei parannusta {epochs_no_improve}/{patience})")

            if epochs_no_improve >= patience:
                print(f"\nEarly stopping {patience} epochin jälkeen ilman parannusta.")
                break

        except Exception as e_valid_epoch:
             print(f"\nVIRHE validointiloopissa (Epoch {epoch+1}, Batch {valid_batch_count}): {e_valid_epoch}")
             traceback.print_exc()
             print("Keskeytetään koulutus.")
             model = None; return model, train_losses, valid_losses

    # --- Koulutusloopin jälkeen ---
    if best_model_state:
         print("\nLadataan paras malli Early Stoppingin perusteella.")
         try: model.load_state_dict(best_model_state)
         except Exception as e_load: print(f"VIRHE parhaan mallin latauksessa: {e_load}. Jatketaan viimeisimmällä.")
    elif epochs > 0 and train_losses is not None : print("\nKoulutus päättyi ilman Early Stoppingia. Käytetään viimeisintä mallia.")
    else: print("\nKoulutusta ei ajettu / keskeytyi.")

    return model, train_losses, valid_losses

print("\nOsa 5: Koulutusfunktion Määrittely (LSTM v1) - OK")

In [ ]:
# @title 6. Mallin Koulutus (Suoritus) (LSTM v1)

# Varmistetaan tuonnit solun itsenäistä ajoa varten
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import traceback
# Varmistetaan myös malliluokka ja koulutusfunktio
if 'LSTMModel' not in locals(): print("VAROITUS: LSTMModel-luokkaa ei määritelty (Aja Osa 4)"); LSTMModel = None
if 'train_model' not in locals(): print("VAROITUS: train_model-funktiota ei määritelty (Aja Osa 5)"); train_model = None

print("--- Aloitetaan Mallin Koulutus (Suoritus) ---")

# Alustetaan model Noneksi siltä varalta että koulutus ei käynnisty tai epäonnistuu
model = None # Käytetään geneeristä nimeä koulutuksen ajaksi
model_lstm = None # Tähän tallennetaan onnistuneesti koulutettu malli
train_losses = None
valid_losses = None

# Varmistetaan, että kaikki tarvittavat muuttujat edellisistä osista ovat olemassa
required_vars_exist = True
vars_to_check = ['train_loader', 'valid_loader', 'INPUT_SIZE', 'device',
                 'RNN_HYPERPARAMS', 'TRAIN_HYPERPARAMS',
                 'LSTMModel', 'train_model']
missing_vars = []
for var in vars_to_check:
    if var not in locals() or (locals()[var] is None and var not in ['train_losses', 'valid_losses', 'model', 'model_lstm']): # Sallitaan näiden olla None aluksi
        missing_vars.append(var)
        required_vars_exist = False

if required_vars_exist:
    # --- Mallin, häviöfunktion ja optimoijan alustus ---
    # Haetaan parametrit sanakirjoista
    try:
        model_type = RNN_HYPERPARAMS.get('model_type', 'LSTM') # Varmistetaan että LSTM ajetaan
        input_size = INPUT_SIZE
        hidden_size = RNN_HYPERPARAMS['hidden_size']
        num_layers = RNN_HYPERPARAMS['num_layers']
        output_size = RNN_HYPERPARAMS['output_size']
        dropout_prob = RNN_HYPERPARAMS['dropout_prob']

        batch_size = TRAIN_HYPERPARAMS['batch_size']
        learning_rate = TRAIN_HYPERPARAMS['learning_rate']
        epochs = TRAIN_HYPERPARAMS['epochs']
        patience = TRAIN_HYPERPARAMS['patience']

        print(f"Varmistetaan ajettava mallityyppi: {model_type}")
        if model_type != 'LSTM':
             print(f"VAROITUS: RNN_HYPERPARAMS['model_type'] ei ole 'LSTM', mutta jatketaan LSTM:llä tässä solussa.")
             # Voitaisiin myös nostaa virhe tai vaihtaa mallia dynaamisesti

        # Alustetaan LSTMModel (varmista että luokka on määritelty)
        if LSTMModel is not None:
            model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout_prob).to(device)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            print("\n--- LSTM-Malli (ennen koulutusta) ---")
            print(model)
            print(f"Ominaisuuksien määrä (Input size): {input_size}")
            param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
            print(f"Koulutettavien parametrien määrä: {param_count}")
            print(f"Käytettävä laite: {device}")
            print("-------------------------------------\n")

            # --- Koulutetaan malli ---
            # Varmista että train_model on määritelty
            if train_model is not None:
                try:
                    # Kutsutaan train_model -funktiota
                    model, train_losses, valid_losses = train_model(
                        model, train_loader, valid_loader, criterion, optimizer, epochs, device, patience
                    )

                    if model is None or train_losses is None or valid_losses is None:
                         print("\nKoulutus keskeytyi tai epäonnistui train_model-funktiossa.")
                         model_lstm = None
                    else:
                         print("\nKoulutus suoritettu.")
                         model_lstm = model # Tallenna onnistunut malli omaan muuttujaan

                         # --- Piirretään häviökäyrät ---
                         if train_losses and valid_losses:
                            plt.figure(figsize=(10, 5))
                            plt.plot(train_losses, label='Training Loss')
                            plt.plot(valid_losses, label='Validation Loss')
                            plt.title('Training and Validation Loss (LSTM)')
                            plt.xlabel('Epoch')
                            plt.ylabel('Loss (MSE)')
                            try:
                                min_loss_val = min(min(train_losses, default=1.0), min(valid_losses, default=1.0))
                                if min_loss_val > 1e-9: plt.yscale('log'); print("Käytetään logaritmista skaalaa.")
                                else: print("Käytetään lineaarista skaalaa (min loss <= 0).")
                            except: print("Käytetään lineaarista skaalaa."); plt.yscale('linear')
                            plt.legend(); plt.grid(True, linestyle=':', alpha=0.7); plt.show()
                         elif train_losses is not None and valid_losses is not None: print("Häviölistat tyhjiä.")
                         else: print("Häviölistoja ei saatu koulutuksesta.")

                except Exception as e:
                    print(f"\nVIRHE train_model-kutsun aikana: {e}")
                    traceback.print_exc()
                    model_lstm = None
            else:
                print("VIRHE: train_model funktiota ei ole määritelty (Aja Osa 5).")
                model_lstm = None

        else:
             print("VIRHE: LSTMModel luokkaa ei ole määritelty (Aja Osa 4).")
             model_lstm = None


    except (KeyError, AssertionError, ValueError) as e_init:
         print(f"\nVIRHE mallin/parametrien alustuksessa: {e_init}")
         traceback.print_exc()
         model_lstm = None

else:
    print(f"\nKoulutusta ei voida aloittaa, koska yksi tai useampi tarvittava muuttuja/funktio puuttuu: {missing_vars}")
    model_lstm = None


# Tulostetaan lopputilanne
if model_lstm is not None:
     print("\nOsa 6: Koulutuksen suoritus (LSTM v1) - VALMIS")
else:
     print("\nOsa 6: Koulutuksen suoritus (LSTM v1) - EPÄONNISTUI / OHITETTIIN")

In [ ]:
# @title 7. Arviointifunktioiden Määrittely (LSTM v1)

import numpy as np
import pandas as pd
# Varmistetaan tuonnit
try:
    from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
    from sklearn.preprocessing import StandardScaler
    import torch
    from torch.utils.data import DataLoader
    from tqdm.notebook import tqdm
    import traceback
except ImportError as e:
    print(f"VIRHE: Tarvittavaa kirjastoa ei löydy: {e}")
    # Estä funktion määrittely, jos tuonnit epäonnistuvat
    raise ImportError("Tarvittavia kirjastoja puuttuu arviointia varten.")


print("--- Määritellään arviointifunktiot ---")

# Funktio baseline-ennusteen laskemiseen
def calculate_baseline_persistence_pytorch(targets_original_np, prediction_horizon):
    """Laskee naiivin persistenssi-baselinen PyTorch-datarakenteelle."""
    print("Lasketaan Baseline-ennuste (jakson eka arvo toistuu)...")
    try:
        if not isinstance(targets_original_np, np.ndarray): raise TypeError("targets_original_np pitää olla numpy array")
        if targets_original_np.ndim != 3 or targets_original_np.shape[-1] != 1:
             # Yritä korjata jos muoto (samples, horizon)
             if targets_original_np.ndim == 2 and targets_original_np.shape[1] == prediction_horizon:
                  print(f"Muutetaan baseline-syötteen muoto {targets_original_np.shape} -> 3D")
                  targets_original_np = targets_original_np[..., np.newaxis]
             else:
                  raise ValueError(f"Odotettiin 3D-muotoa (samples, horizon, 1), saatiin {targets_original_np.shape}")

        if targets_original_np.shape[0] == 0: return np.zeros((0, prediction_horizon))
        first_vals = targets_original_np[:, 0, 0]; first_vals_for_repeat = first_vals[:, np.newaxis]
        baseline_preds = np.repeat(first_vals_for_repeat, prediction_horizon, axis=1)
        return baseline_preds
    except Exception as e_base:
        print(f"VIRHE baseline-laskennassa: {e_base}"); traceback.print_exc()
        try: return np.zeros((targets_original_np.shape[0], prediction_horizon))
        except: return None

# Funktio mallin suorituskyvyn arviointiin
def evaluate_model_performance_pytorch(model, test_loader, device, o3_scaler, o3_threshold_8h, prediction_horizon):
    """Arvioi PyTorch-mallia testidatalla, laskee metriikat ja vertaa baselineen."""
    print("\n--- evaluate_model_performance_pytorch -funktion suoritus alkaa ---")
    if model is None: print("Malli puuttuu."); return None, None
    if test_loader is None: print("Testilataaja puuttuu."); return None, None
    if o3_scaler is None: print("O3-skaalain puuttuu."); return None, None

    model.eval(); all_preds_orig_list = []; all_targets_orig_list = []
    print("Aloitetaan ennusteiden tekeminen testidatalla...")
    try:
        with torch.no_grad():
            for inputs, targets_original_batch in tqdm(test_loader, desc="Testaus (evaluate)"):
                inputs = inputs.to(device)
                outputs_scaled = model(inputs)
                preds_scaled_np = outputs_scaled.cpu().numpy()
                preds_reshaped = preds_scaled_np.reshape(-1, 1)
                preds_orig_np = o3_scaler.inverse_transform(preds_reshaped).reshape(preds_scaled_np.shape)
                all_preds_orig_list.append(preds_orig_np)
                all_targets_orig_list.append(targets_original_batch.cpu().numpy())
        print("Ennusteiden tekeminen ja kerääminen valmis.")
        if not all_preds_orig_list or not all_targets_orig_list: raise ValueError("Ennusteiden/kohteiden keräys epäonnistui.")
        all_preds_orig = np.concatenate(all_preds_orig_list, axis=0)
        all_targets_original = np.concatenate(all_targets_orig_list, axis=0) # Muoto (samples, horizon, 1)
        print(f"Kerätty {all_preds_orig.shape[0]} ennustetta/kohdetta.")

        # Poista viimeinen dimensio kohteista
        if all_targets_original.ndim == 3 and all_targets_original.shape[-1] == 1:
            targets_eval = all_targets_original.squeeze(-1)
        elif all_targets_original.ndim == 2: targets_eval = all_targets_original
        else: raise ValueError(f"Odottamaton kohdemuoto metriikoille: {all_targets_original.shape}")
        print(f"Muutettu kohdemuoto metriikoita varten: {targets_eval.shape}")
        if all_preds_orig.shape != targets_eval.shape: raise ValueError(f"Lopulliset muodot eivät täsmää: Pred={all_preds_orig.shape}, Target={targets_eval.shape}")

        # --- Laske regressiometriikat (Nykyinen malli) ---
        print("\nLasketaan nykyisen mallin regressiometriikat...")
        # Käytä .ravel() laskeaksesi virheen kaikista pisteistä
        rmse_model = np.sqrt(mean_squared_error(targets_eval.ravel(), all_preds_orig.ravel()))
        mae_model = mean_absolute_error(targets_eval.ravel(), all_preds_orig.ravel())
        print(f"\n--- Nykyisen Mallin ({type(model).__name__}) Arviointi ---")
        print(f"RMSE: {rmse_model:.4f} µg/m³")
        print(f"MAE:  {mae_model:.4f} µg/m³")

        # --- Laske Baseline ---
        baseline_preds = calculate_baseline_persistence_pytorch(all_targets_original, prediction_horizon) # Käyttää 3D-muotoa
        rmse_baseline, mae_baseline = None, None # Alustus
        if baseline_preds is not None and baseline_preds.shape == targets_eval.shape:
            if np.isnan(baseline_preds).any(): print("VAROITUS: Baseline sisältää NaN. Täytetään."); baseline_preds = pd.DataFrame(baseline_preds).ffill().bfill().values
            if not np.isnan(baseline_preds).any():
                rmse_baseline = np.sqrt(mean_squared_error(targets_eval.ravel(), baseline_preds.ravel()))
                mae_baseline = mean_absolute_error(targets_eval.ravel(), baseline_preds.ravel())
                print(f"\n--- Baseline-Mallin Arviointi ---")
                print(f"RMSE: {rmse_baseline:.4f} µg/m³")
                print(f"MAE:  {mae_baseline:.4f} µg/m³")
            else: print("Baseline-metriikoita ei laskettu (NaN).")
        else: print("Baseline-ennustetta ei laskettu tai muoto väärä.")

        # --- Vertailu ---
        print("\n--- Vertailu Baselineen ---")
        if rmse_model is not None and rmse_baseline is not None: print(f"Malli vs Baseline RMSE: {rmse_baseline - rmse_model:+.4f} µg/m³ ({'Malli parempi' if (rmse_baseline - rmse_model) > 0 else 'Baseline parempi/sama'})")
        else: print("RMSE-vertailua ei voida tehdä.")
        if mae_model is not None and mae_baseline is not None: print(f"Malli vs Baseline MAE:  {mae_baseline - mae_model:+.4f} µg/m³ ({'Malli parempi' if (mae_baseline - mae_model) > 0 else 'Baseline parempi/sama'})")
        else: print("MAE-vertailua ei voida tehdä.")

        # --- 8h Liukuvan keskiarvon arviointi ---
        warnings_actual = []; warnings_pred = []
        n_samples_total = all_preds_orig.shape[0]
        print(f"\nLasketaan 8h liukuvia keskiarvoja {n_samples_total} jaksolle (kynnys={o3_threshold_8h} µg/m³)...")
        for i in range(n_samples_total):
            actual_series = pd.Series(targets_eval[i, :]); pred_series = pd.Series(all_preds_orig[i, :])
            actual_8h_avg = actual_series.rolling(window=8, min_periods=1).mean()
            pred_8h_avg = pred_series.rolling(window=8, min_periods=1).mean()
            warnings_actual.append(actual_8h_avg.max() > o3_threshold_8h)
            warnings_pred.append(pred_8h_avg.max() > o3_threshold_8h)
        warnings_actual = np.array(warnings_actual); warnings_pred = np.array(warnings_pred)
        print("Liukuvien keskiarvojen laskenta valmis.")

        print(f"\n--- 8h Liukuvan Keskiarvon Varoitustason Ylityksen Arviointi ({type(model).__name__} - Jaksoittain) ---")
        print(f"Todellisia varoitusjaksoja (> {o3_threshold_8h}): {warnings_actual.sum()} / {n_samples_total}")
        print(f"Ennustettuja varoitusjaksoja ({type(model).__name__}): {warnings_pred.sum()} / {n_samples_total}")
        if warnings_actual.sum() > 0 or warnings_pred.sum() > 0:
            print("\nSekaannusmatriisi:"); cm = confusion_matrix(warnings_actual, warnings_pred, labels=[False, True])
            print(pd.DataFrame(cm, index=['Todellinen EI', 'Todellinen KYLLÄ'], columns=['Ennuste EI', 'Ennuste KYLLÄ']))
            print("\nLuokitteluraportti:"); report = classification_report(warnings_actual, warnings_pred, target_names=['Ei Varoitusta', 'Varoitus'], labels=[False, True], zero_division=0); print(report)
            TN, FP, FN, TP = cm.ravel(); recall = TP/(TP+FN) if (TP+FN)>0 else 0; precision = TP/(TP+FP) if (TP+FP)>0 else 0
            print(f"---> Recall: {recall:.4f}, Precision: {precision:.4f}")
        else: print(f"Ei todellisia eikä ennustettuja varoitusjaksoja kynnysarvolla {o3_threshold_8h}.")

        print("\n--- evaluate_model_performance_pytorch -funktion suoritus päättyi onnistuneesti ---")
        return all_preds_orig, targets_eval # Palautetaan 2D muodot

    except Exception as e:
        print(f"\n-----> VIRHE evaluate_model_performance_pytorch -FUNKTIOSSA <-----")
        traceback.print_exc(); return None, None

print("\nOsa 7: Arviointifunktioiden Määrittely (LSTM v1) - OK")

In [ ]:
# @title 8. Mallin Arviointi (Suoritus) (LSTM v1)

import traceback
import numpy as np # Varmistetaan tuonti
import pandas as pd # Varmistetaan tuonti

print("--- Aloitetaan Mallin Arviointi (Suoritus) ---")

# Alustetaan tulosmuuttujat
test_preds_lstm = None
test_targets_lstm = None
evaluation_lstm_successful = False # Lipuke onnistumiselle

# Varmistetaan, että tarvittavat muuttujat edellisistä osista ovat olemassa
# Käytetään nyt model_lstm muuttujaa, johon tallennettiin koulutettu malli
required_eval_vars = ['model_lstm', 'test_loader', 'device', 'o3_scaler',
                      'O3_THRESHOLD_8H_AVG', 'PREDICTION_HORIZON',
                      'evaluate_model_performance_pytorch']
missing_eval_vars = []
for var in required_eval_vars:
     if var not in locals() or locals()[var] is None:
          missing_eval_vars.append(var)

if not missing_eval_vars:
    # --- Suoritetaan arviointi ---
    try:
        print(f"\nKutsutaan evaluate_model_performance_pytorch LSTM-mallille (kynnys={O3_THRESHOLD_8H_AVG})...")
        # Annetaan koulutettu LSTM-malli funktiolle
        # Palauttaa 2D-muotoiset ennusteet ja kohteet
        test_preds_lstm, test_targets_lstm = evaluate_model_performance_pytorch(
            model_lstm, # Käytä koulutettua LSTM-mallia
            test_loader,
            device,
            o3_scaler,
            O3_THRESHOLD_8H_AVG,
            PREDICTION_HORIZON
        )

        # Tarkistetaan paluuarvot
        if test_preds_lstm is not None and test_targets_lstm is not None:
            print("\nArviointifunktion ajo suoritettu.")
            if isinstance(test_preds_lstm, np.ndarray) and isinstance(test_targets_lstm, np.ndarray) \
               and test_preds_lstm.ndim == 2 and test_targets_lstm.ndim == 2 \
               and test_preds_lstm.shape == test_targets_lstm.shape:
                 print(f"Arviointi palautti validit tulokset muodoissa: Preds={test_preds_lstm.shape}, Targets={test_targets_lstm.shape}")
                 evaluation_lstm_successful = True
            else:
                 print("VIRHE: Arviointifunktion palauttamat arvot eivät ole odotettuja numpy arrayta tai muodot eivät täsmää.")
                 test_preds_lstm, test_targets_lstm = None, None

        else:
            print("\nArviointi epäonnistui (funktio palautti None).")

    except Exception as e:
        print(f"\nVIRHE arvioinnin suorituksessa: {e}")
        traceback.print_exc()
        test_preds_lstm, test_targets_lstm = None, None

else:
    print(f"\nArviointia ei voida suorittaa, koska yksi tai useampi tarvittava muuttuja puuttuu: {missing_eval_vars}")
    test_preds_lstm, test_targets_lstm = None, None


# Tulostetaan lopputilanne
if evaluation_lstm_successful:
     print("\nOsa 8: Arvioinnin suoritus (LSTM v1) - VALMIS")
else:
     print("\nOsa 8: Arvioinnin suoritus (LSTM v1) - EPÄONNISTUI / OHITETTIIN")